In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

from lightning.pytorch.tuner import Tuner


In [2]:
# setup data
DATASETS_PATH = os.path.join("..", "..", "datasets")
print('DATASETS_PATH: ', DATASETS_PATH)

dataset = MNIST(DATASETS_PATH, download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

DATASETS_PATH:  ..\..\datasets


In [3]:


# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder, batch_size=32, train_loader=None):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.batch_size = batch_size
        self.train_loader = train_loader

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def train_dataloader(self):
        return self.train_loader


# init the autoencoder
batch_size = 32
autoencoder = LitAutoEncoder(encoder, decoder, batch_size, train_loader)

In [4]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
# https://lightning.ai/docs/pytorch/stable/common/trainer.html
# https://lightning.ai/docs/pytorch/stable/advanced/training_tricks.html

trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
# print('trainer: ', help(trainer))

# Create a tuner for the trainer
tuner = Tuner(trainer)
print('autoencoder.batch_size: ', autoencoder.batch_size)

# Auto-scale batch size by growing it exponentially (default)
tuner.scale_batch_size(model=autoencoder, mode="power")
print('autoencoder.batch_size: ', autoencoder.batch_size)

# Auto-scale batch size with binary search
tuner.scale_batch_size(model=autoencoder, mode="binsearch")
print('autoencoder.batch_size: ', autoencoder.batch_size)

trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


autoencoder.batch_size:  32


Missing logger folder: C:\Users\GyanT\Documents\GitHub\Artificial-Neural-Network\PyTorchLightning\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\GyanT\miniconda3\envs\py3_10\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 2 succeeded, trying batch size 4
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 8 succeeded, trying batch size 16
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 16 succeeded, trying batch size 32
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 32 succeeded, trying

autoencoder.batch_size:  65536


`Trainer.fit` stopped: `max_steps=3` reached.
The batch size 65536 is greater or equal than the length of your dataset.
Finished batch size finder, will continue with full run using batch size 65536
Restoring states from the checkpoint path at C:\Users\GyanT\Documents\GitHub\Artificial-Neural-Network\PyTorchLightning\.scale_batch_size_6c8e5620-87db-4696-9568-5ab58b5c5dfe.ckpt
Restored all states from the checkpoint at C:\Users\GyanT\Documents\GitHub\Artificial-Neural-Network\PyTorchLightning\.scale_batch_size_6c8e5620-87db-4696-9568-5ab58b5c5dfe.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


autoencoder.batch_size:  65536


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28).to(device='cuda')
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[0.0088, 0.1069, 0.0755],
        [0.0088, 0.1069, 0.0755],
        [0.0088, 0.1069, 0.0755],
        [0.0088, 0.1069, 0.0755]], device='cuda:0', grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
